# Самолеты-самолеты

In [16]:
import numpy as np
import pandas as pd

In [17]:
airports = pd.read_csv('airports.csv',names = ['Airport ID','Name','City','Country','IATA','ICAO','Latitude','Longitude','Altitude','Timezone','DST','Tz_database_time_zone','Type','Source'])
routes = pd.read_csv('routes.csv',names = ['Airline','Airport ID','Source airport','Source airport ID','Destination airport','Destination airport ID','Codeshare','Stops','Equipment'])

Перечень стран Европы:

In [18]:
europe=np.array(['Belgium', 'Germany', 'Estonia', 'Finland', 'United Kingdom',
'Guernsey', 'Jersey', 'Isle of Man', 'Netherlands', 'Ireland',
'Denmark', 'Luxembourg', 'Norway', 'Poland', 'Sweden', 'Spain',
'Albania', 'Bulgaria', 'Cyprus', 'Croatia', 'France', 'Greece',
'Hungary', 'Italy', 'Slovenia', 'Czech Republic', 'Malta',
'Austria', 'Portugal', 'Bosnia and Herzegovina', 'Romania',
'Switzerland', 'Turkey', 'Moldova', 'Macedonia', 'Gibraltar',
'Serbia', 'Montenegro', 'Slovakia', 'Ukraine', 'Russia', 'Belarus',
'Latvia', 'Lithuania','Faroe Islands', 'Greenland', 'Iceland'])

Отберем аэропорты для стран Европы имеющих трех буквенный код IATA:

In [19]:
airports = airports[(airports['IATA'].str.len() == 3) & (airports['Country'].isin(europe))]

Пометим пропущенные данные, как ``np.nan`` 

In [20]:
airports = airports.replace('\\N', np.nan)
routes = routes.replace('\\N', np.nan)

Посчитать взлет/посадки для каждого аэропорта стран Европы

In [21]:
sour = routes.groupby(routes['Source airport']).count()[['Source airport ID']]
dest = routes.groupby(routes['Destination airport']).count()[['Destination airport ID']]

sour['IATA'] = sour.index
dest['IATA'] = dest.index

In [22]:
result = sour.merge(dest,left_on='IATA', right_on='IATA' , how='inner' )  # Если нет зачения взлет или посадка, аэропорт выкидываем из списка - inner join. 
result['Fly'] =  result['Source airport ID'] + result['Destination airport ID'] 

In [23]:
airports = airports.merge(result,left_on='IATA', right_on='IATA' , how='inner' ).set_index('Name')

Отсортируем по названию страны, по значению Fly, возьмем 3 значения:

In [27]:
answer = airports.groupby('Country')['Fly'].nlargest(3)

Ответ:

In [28]:
answer.to_frame(name='Fly').head()

Fly
Country Name                                           
Albania Tirana International Airport Mother Teresa   72
Austria Vienna International Airport                616
        Salzburg Airport                             69
        Graz Airport                                 38
Belarus Minsk National Airport                      108